In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [3]:
def standardize(train, test):
    mean, std = train.mean(), train.std()
    train = (train - mean) / std
    test = (test - mean) / std
    return train, test

In [4]:
x_train, x_test = standardize(x_train, x_test)



In [5]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['elu']

In [7]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': keras.layers.ELU(alpha=1.5),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.0) 
        }

        model_name = 'std_' + act + '_1.5_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation elu with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.4001 - acc: 0.8791 - val_loss: 0.1878 - val_acc: 0.9438
Epoch 2/100
60000/60000 [==============================] - 3s 45us/step - loss: 0.2210 - acc: 0.9333 - val_loss: 0.1192 - val_acc: 0.9644
Epoch 3/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.1780 - acc: 0.9464 - val_loss: 0.1076 - val_acc: 0.9680
Epoch 4/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.1536 - acc: 0.9536 - val_loss: 0.0981 - val_acc: 0.9711
Epoch 5/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1404 - acc: 0.9582 - val_loss: 0.0916 - val_acc: 0.9734
Epoch 6/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.1282 - acc: 0.9617 - val_loss: 0.0977 - val_acc: 0.9709
Epoch 7/100
60000/60000 [==============================] - 2s 39us/step

60000/60000 [==============================] - 2s 38us/step - loss: 0.0657 - acc: 0.9812 - val_loss: 0.1112 - val_acc: 0.9788
Epoch 60/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0610 - acc: 0.9823 - val_loss: 0.1135 - val_acc: 0.9782
Epoch 61/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0646 - acc: 0.9814 - val_loss: 0.1070 - val_acc: 0.9789
Epoch 62/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0629 - acc: 0.9829 - val_loss: 0.1228 - val_acc: 0.9753
Epoch 63/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.0628 - acc: 0.9823 - val_loss: 0.1121 - val_acc: 0.9764
Epoch 64/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0672 - acc: 0.9819 - val_loss: 0.1241 - val_acc: 0.9770
Epoch 65/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0651 - acc: 0.9823 - val_loss: 0.1101 - val_acc: 0.9777
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0919 - acc: 0.9733 - val_loss: 0.0852 - val_acc: 0.9768
Epoch 18/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0887 - acc: 0.9733 - val_loss: 0.0834 - val_acc: 0.9776
Epoch 19/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0860 - acc: 0.9747 - val_loss: 0.0934 - val_acc: 0.9767
Epoch 20/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0861 - acc: 0.9748 - val_loss: 0.0902 - val_acc: 0.9773
Epoch 21/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0850 - acc: 0.9750 - val_loss: 0.0871 - val_acc: 0.9767
Epoch 22/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0828 - acc: 0.9748 - val_loss: 0.0908 - val_acc: 0.9771
Epoch 23/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0812 - acc: 0.9758 - val_loss: 0.0910 - val_acc: 0.9759
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0641 - acc: 0.9827 - val_loss: 0.1140 - val_acc: 0.9771
Epoch 77/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0635 - acc: 0.9819 - val_loss: 0.1140 - val_acc: 0.9757
Epoch 78/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0635 - acc: 0.9821 - val_loss: 0.1030 - val_acc: 0.9778
Epoch 79/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0632 - acc: 0.9829 - val_loss: 0.1084 - val_acc: 0.9788
Epoch 80/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0659 - acc: 0.9817 - val_loss: 0.1150 - val_acc: 0.9773
Epoch 81/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0623 - acc: 0.9828 - val_loss: 0.1169 - val_acc: 0.9769
Epoch 82/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0637 - acc: 0.9822 - val_loss: 0.1147 - val_acc: 0.9773
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0681 - acc: 0.9795 - val_loss: 0.0874 - val_acc: 0.9784
Epoch 35/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0684 - acc: 0.9802 - val_loss: 0.0928 - val_acc: 0.9797
Epoch 36/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0677 - acc: 0.9795 - val_loss: 0.0924 - val_acc: 0.9790
Epoch 37/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0702 - acc: 0.9797 - val_loss: 0.0959 - val_acc: 0.9799
Epoch 38/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.0700 - acc: 0.9799 - val_loss: 0.0903 - val_acc: 0.9794
Epoch 39/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.0672 - acc: 0.9806 - val_loss: 0.0983 - val_acc: 0.9791
Epoch 40/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0666 - acc: 0.9808 - val_loss: 0.0949 - val_acc: 0.9801
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.0590 - acc: 0.9839 - val_loss: 0.1237 - val_acc: 0.9788
Epoch 94/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0598 - acc: 0.9841 - val_loss: 0.1166 - val_acc: 0.9788
Epoch 95/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0618 - acc: 0.9832 - val_loss: 0.1232 - val_acc: 0.9777
Epoch 96/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0635 - acc: 0.9836 - val_loss: 0.1132 - val_acc: 0.9792
Epoch 97/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0607 - acc: 0.9834 - val_loss: 0.1319 - val_acc: 0.9784
Epoch 98/100
60000/60000 [==============================] - 3s 42us/step - loss: 0.0592 - acc: 0.9846 - val_loss: 0.1292 - val_acc: 0.9792
Epoch 99/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0603 - acc: 0.9834 - val_loss: 0.1249 - val_acc: 0.9774
Epoch 100/100
60000/60000 [

Epoch 51/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0624 - acc: 0.9815 - val_loss: 0.1061 - val_acc: 0.9765
Epoch 52/100
60000/60000 [==============================] - 3s 42us/step - loss: 0.0662 - acc: 0.9810 - val_loss: 0.1057 - val_acc: 0.9770
Epoch 53/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0656 - acc: 0.9811 - val_loss: 0.1059 - val_acc: 0.9794
Epoch 54/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0641 - acc: 0.9815 - val_loss: 0.1074 - val_acc: 0.9779
Epoch 55/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0652 - acc: 0.9818 - val_loss: 0.1068 - val_acc: 0.9783
Epoch 56/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0664 - acc: 0.9816 - val_loss: 0.1027 - val_acc: 0.9785
Epoch 57/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0645 - acc: 0.9823 - val_loss: 0.1194 - val_acc: 0.9764
Epoch 58/100
60000/60000 [=

Epoch 9/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.1103 - acc: 0.9667 - val_loss: 0.0824 - val_acc: 0.9755
Epoch 10/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.1031 - acc: 0.9688 - val_loss: 0.0847 - val_acc: 0.9738
Epoch 11/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.1034 - acc: 0.9693 - val_loss: 0.0867 - val_acc: 0.9748
Epoch 12/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.0951 - acc: 0.9712 - val_loss: 0.0846 - val_acc: 0.9769
Epoch 13/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.0954 - acc: 0.9711 - val_loss: 0.0846 - val_acc: 0.9761
Epoch 14/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.0924 - acc: 0.9721 - val_loss: 0.0870 - val_acc: 0.9774
Epoch 15/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.0892 - acc: 0.9727 - val_loss: 0.0788 - val_acc: 0.9796
Epoch 16/100
60000/60000 [==

Epoch 68/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0638 - acc: 0.9826 - val_loss: 0.1137 - val_acc: 0.9786
Epoch 69/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.0617 - acc: 0.9827 - val_loss: 0.1053 - val_acc: 0.9794
Epoch 70/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.0607 - acc: 0.9834 - val_loss: 0.1048 - val_acc: 0.9802
Epoch 71/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.0628 - acc: 0.9825 - val_loss: 0.0989 - val_acc: 0.9777
Epoch 72/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.0621 - acc: 0.9832 - val_loss: 0.1041 - val_acc: 0.9797
Epoch 73/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.0598 - acc: 0.9835 - val_loss: 0.1097 - val_acc: 0.9787
Epoch 74/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0625 - acc: 0.9824 - val_loss: 0.1103 - val_acc: 0.9786
Epoch 75/100
60000/60000 [=